In [62]:
import pandas as pd
import matplotlib.pyplot as plt

import re
import os

# Util Functions

In [63]:
def split(delimiters, string, maxsplit=0):
    regexPattern = '|'.join(map(re.escape, delimiters))
    return re.split(regexPattern, string, maxsplit)

# Data Collection

In [64]:
scalehls_filepath = '../results/scale_hls_summary_results.csv'

In [65]:
jobs_f_name = '../jobs-medium.sh'
with open(jobs_f_name, "r") as f:

  raw_exp_list= []
  for l in  f.readlines():
    if (not('#' in l)):
      if ('code' in l):
        raw_exp_list.append(l.strip('\n'))

  #print(raw_exp_list)

  exp_list =[]
  for e in raw_exp_list:
    r = split(['/','-',' ','.'],e)
    exp_list.append(r)

  # print(exp_list)

  paths_to_results = []
  for v in exp_list:
    tmp_path = '../'+v[0]+'/'+v[1]+'/output/'+v[1]+'_'+v[-1]+'/'+v[6]+'-'+v[8]+'-'+v[9]+'/bambu/results.txt'
    completed = os.path.exists(tmp_path)
    # print(completed, tmp_path)

    cycles=None
    if (completed):
      l = 0
      with open(tmp_path, "r") as f1:
        l=f1.readlines()[0]
        cycles = l.replace(' ',"").strip('\n').split('\t')[-1]
    

    tmp_synt_path = '../'+v[0]+'/'+v[1]+'/output/'+v[1]+'_'+v[-1]+'/'+v[6]+'-'+v[8]+'-'+v[9]+'/bambu/HLS_output/Synthesis/vivado_flow/'+v[1]+'_'+v[-1]+'_kernel_report.xml'
    synthesized = os.path.exists(tmp_synt_path)

    # print(synthesized, tmp_synt_path)

    power=None
    lut=None
    dsp=None
    if (synthesized):
      syn_results = pd.read_xml(tmp_synt_path,  xpath='.//section//item')

      tmp_df = syn_results.pivot_table(index=None, columns='stringID', values='value').reset_index()
      lut=tmp_df['XILINX_SLICE_LUTS'][0]
      dsp=tmp_df['XILINX_DSPS'][0]
      power=tmp_df['XILINX_POWER'][0]
    
    else:
      lut=None
      dsp=None
      power=None
    
    v.append(completed)
    v.append(cycles)
    v.append(synthesized)
    v.append(lut)
    v.append(dsp)
    v.append(power)
          
# exp_list

columns =['path0',
  'kernel_name',
  'path1',
  'path2',
  'path3',
  'outline_strategy',
  'opt_strategy',
  'path4',
  'math_type',
  'has_sdcss',
  'path5',
  'k_size',
  'completed',
  'cycles',
  'synthesized',
  'luts',
  'dsps',
  'power'
  ]

soda_opt_df = pd.DataFrame( exp_list ,columns = columns)
drops = ['path0','path1','path2','path3','path4','path5']
for v in drops:
  soda_opt_df.drop(v, axis=1, inplace=True)

soda_opt_df['cycles']=soda_opt_df['cycles'].astype(float)
soda_opt_df['luts']=soda_opt_df['luts'].astype(float)
soda_opt_df['dsps']=soda_opt_df['dsps'].astype(float)
soda_opt_df['power']=soda_opt_df['power'].astype(float)
soda_opt_df['k_size']=soda_opt_df['k_size'].astype(int)
# df['k_size']=df['k_size'].astype("string")
soda_opt_df

,kernel_name,outline_strategy,opt_strategy,math_type,has_sdcss,k_size,completed,cycles,synthesized,luts,dsps,power
0,bicg,affine_for,opt_full,soft_float,no_ssdcs,2,True,13.0,True,2306.0,16.0,0.371
1,bicg,affine_for,opt_full,soft_float,no_ssdcs,4,True,28.0,True,5834.0,26.0,0.392
2,bicg,affine_for,opt_full,soft_float,no_ssdcs,8,True,45.0,True,15475.0,40.0,0.695
3,bicg,affine_for,opt_full,soft_float,no_ssdcs,16,True,141.0,True,41617.0,54.0,1.250
4,bicg,affine_for,opt_none,soft_float,no_ssdcs,2,True,73.0,True,1370.0,2.0,0.348
...,...,...,...,...,...,...,...,...,...,...,...,...
92,mvt,affine_for,opt_full,soft_float,no_ssdcs,16,True,141.0,True,42507.0,54.0,1.255
93,mvt,affine_for,opt_none,soft_float,no_ssdcs,2,True,74.0,True,1464.0,2.0,0.350
94,mvt,affine_for,opt_none,soft_float,no_ssdcs,4,True,290.0,True,3893.0,2.0,0.385
95,mvt,affine_for,opt_none,soft_float,no_ssdcs,8,True,1155.0,True,2966.0,2.0,0.369


In [66]:
scale_hls_df = pd.read_csv(scalehls_filepath)
scale_hls_df['cycles']=scale_hls_df['cycles'].astype(float)
scale_hls_df['luts']=scale_hls_df['luts'].astype(float)
scale_hls_df['dsps']=scale_hls_df['dsps'].astype(float)
scale_hls_df['power']=scale_hls_df['power'].astype(float)
scale_hls_df['k_size']=scale_hls_df['k_size'].astype(int)


df=pd.concat([soda_opt_df,scale_hls_df])
df['cycles']=df['cycles'].astype(float)
df['luts']=df['luts'].astype(float)
df['dsps']=df['dsps'].astype(float)
df['power']=df['power'].astype(float)
df['k_size']=df['k_size'].astype(int)

with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(df)

# df.to_csv('../results/all.csv')

,kernel_name,outline_strategy,opt_strategy,math_type,has_sdcss,k_size,completed,cycles,synthesized,luts,dsps,power
0,bicg,affine_for,opt_full,soft_float,no_ssdcs,2,True,13.0,True,2306.0,16.0,0.371
1,bicg,affine_for,opt_full,soft_float,no_ssdcs,4,True,28.0,True,5834.0,26.0,0.392
2,bicg,affine_for,opt_full,soft_float,no_ssdcs,8,True,45.0,True,15475.0,40.0,0.695
3,bicg,affine_for,opt_full,soft_float,no_ssdcs,16,True,141.0,True,41617.0,54.0,1.250
4,bicg,affine_for,opt_none,soft_float,no_ssdcs,2,True,73.0,True,1370.0,2.0,0.348
5,bicg,affine_for,opt_none,soft_float,no_ssdcs,4,True,294.0,True,1642.0,2.0,0.352
6,bicg,affine_for,opt_none,soft_float,no_ssdcs,8,True,1162.0,True,2614.0,2.0,0.366
7,bicg,affine_for,opt_none,soft_float,no_ssdcs,16,True,4626.0,True,1027.0,2.0,0.338
8,gemm,affine_for,opt_full,soft_float,no_ssdcs,2,True,16.0,True,2537.0,16.0,0.377
9,gemm,affine_for,opt_full,soft_float,no_ssdcs,4,True,28.0,True,11865.0,72.0,0.468


# Data Processing

In [67]:
print("\nCycles")
cycles_df = df.pivot(index='kernel_name', columns=[ 'opt_strategy', 'k_size'], values='cycles').sort_index(axis='columns', level=['opt_strategy','k_size'])
display(cycles_df)

print("\nLUTs")
luts_df = df.pivot(index='kernel_name', columns=[ 'opt_strategy', 'k_size'], values='luts').sort_index(axis='columns', level=['opt_strategy','k_size'])
display(luts_df)

print("\nDSPs")
dsps_df = df.pivot(index='kernel_name', columns=[ 'opt_strategy', 'k_size'], values='dsps').sort_index(axis='columns', level=['opt_strategy','k_size'])
display(dsps_df)

print("\nPower")
power_df = df.pivot(index='kernel_name', columns=[ 'opt_strategy', 'k_size'], values='power').sort_index(axis='columns', level=['opt_strategy','k_size'])
display(power_df)

all_dfs = [cycles_df, luts_df, dsps_df, power_df]
all_dfs_names = ['cycles_df', 'luts_df', 'dsps_df', 'power_df']

# Uncomment to save all
# with pd.ExcelWriter('../results/all.xlsx') as writer: 
#   for d,n in zip(all_dfs,all_dfs_names):
#     d.to_excel(writer, sheet_name=n)


Cycles


opt_strategy opt_full                        opt_nbt                          \
k_size             2      4       8       16      2      4       8        16   
kernel_name                                                                    
atax             21.0   34.0    60.0   157.0     NaN    NaN     NaN      NaN   
bicg             13.0   28.0    45.0   141.0     NaN    NaN     NaN      NaN   
doitgen          15.0  166.0  1158.0     NaN     NaN    NaN     NaN      NaN   
gemm             16.0   28.0    71.0     NaN     NaN    NaN     NaN      NaN   
gemver           36.0   49.0    75.0     NaN     NaN    NaN     NaN      NaN   
gesummv          18.0   25.0    46.0   142.0     NaN    NaN     NaN      NaN   
mvt              13.0   21.0    45.0   141.0     NaN    NaN     NaN      NaN   
symm              NaN    NaN     NaN     NaN    15.0   18.0   267.0      NaN   
syr2k            19.0  270.0  1417.0  8835.0     NaN    NaN     NaN      NaN   
syrk             16.0  222.0  1211.0  7896.0     NaN    NaN     NaN      NaN   
three_mm         22.0   40.0     NaN     NaN     NaN    NaN     NaN      NaN   
trmm              NaN    NaN     NaN     NaN    12.0  288.0  2138.0  15730.0   
two_mm           25.0   43.0    98.0     NaN     NaN    NaN     NaN      NaN   

opt_strategy opt_none                            scalehls                       
k_size             2       4        8         16       2     4      8       16  
kernel_name                                                                     
atax             76.0   299.0   1171.0    4643.0      NaN   NaN    NaN  1701.0  
bicg             73.0   294.0   1162.0    4626.0     26.0  43.0   78.0   151.0  
doitgen         165.0  2486.0  38986.0  344338.0     19.0  33.0   96.0    96.0  
gemm            103.0   794.0   6538.0   42514.0     23.0  33.0   64.0   146.0  
gemver          154.0   606.0   2377.0    9620.0      NaN   NaN    NaN   242.0  
gesummv          92.0   326.0   1226.0    5026.0     28.0  38.0   65.0   111.0  
mvt              74.0   290.0   1155.0    4611.0      NaN   NaN    NaN   112.0  
symm            229.0  1593.0      NaN   90133.0      NaN   NaN    NaN     NaN  
syr2k            99.0   706.0   4834.0   35650.0     35.0  47.0   81.0   381.0  
syrk             79.0   546.0   3682.0   22594.0     28.0  39.0   74.0   340.0  
three_mm        220.0  1743.0  14042.0  111410.0      NaN   NaN  188.0   827.0  
trmm             26.0   301.0   2138.0   15730.0     40.0  84.0  182.0  1199.0  
two_mm          176.0  1375.0  11218.0   87842.0      NaN   NaN  124.0   788.0


LUTs


opt_strategy opt_full                             opt_nbt                   \
k_size             2        4         8        16      2       4        8    
kernel_name                                                                  
atax           1423.0   4433.0   14973.0  42007.0     NaN     NaN      NaN   
bicg           2306.0   5834.0   15475.0  41617.0     NaN     NaN      NaN   
doitgen        4152.0  10992.0   64250.0      NaN     NaN     NaN      NaN   
gemm           2537.0  11865.0   59841.0      NaN     NaN     NaN      NaN   
gemver         3428.0  13571.0   36063.0      NaN     NaN     NaN      NaN   
gesummv        2629.0   6218.0   15207.0  42428.0     NaN     NaN      NaN   
mvt            2294.0   5725.0   15503.0  42507.0     NaN     NaN      NaN   
symm              NaN      NaN       NaN      NaN  1762.0  4164.0  56782.0   
syr2k          3255.0   3080.0    8212.0  30009.0     NaN     NaN      NaN   
syrk           2081.0   1488.0    4003.0  14641.0     NaN     NaN      NaN   
three_mm       5243.0  28910.0       NaN      NaN     NaN     NaN      NaN   
trmm              NaN      NaN       NaN      NaN  1198.0  1726.0    831.0   
two_mm         3184.0  19370.0  110513.0      NaN     NaN     NaN      NaN   

opt_strategy        opt_none                         scalehls           \
k_size           16       2       4       8       16       2        4    
kernel_name                                                              
atax            NaN   1371.0  1764.0  2668.0  4775.0      NaN      NaN   
bicg            NaN   1370.0  1642.0  2614.0  1027.0    989.0   2022.0   
doitgen         NaN   2300.0  3496.0  2824.0  2382.0   6113.0  38699.0   
gemm            NaN   1929.0  3385.0  2693.0  1005.0   4663.0  29085.0   
gemver          NaN   2728.0  6182.0  6584.0  8509.0      NaN      NaN   
gesummv         NaN   1690.0  1838.0  2883.0  1197.0   4347.0  14041.0   
mvt             NaN   1464.0  3893.0  2966.0  5461.0      NaN      NaN   
symm            NaN   2802.0  2645.0  4356.0  1501.0      NaN      NaN   
syr2k           NaN   2049.0  2440.0  4147.0  1294.0   1607.0   4831.0   
syrk            NaN   1403.0  1530.0  2626.0  1040.0   1930.0   4957.0   
three_mm        NaN   3122.0  7108.0  7696.0  2014.0      NaN      NaN   
trmm          839.0    975.0  2517.0  1010.0  1005.0   3278.0  18561.0   
two_mm          NaN   2917.0  5700.0  5275.0  1641.0      NaN      NaN   

opt_strategy                    
k_size             8        16  
kernel_name                     
atax              NaN   7073.0  
bicg           3559.0   8393.0  
doitgen       62305.0  88450.0  
gemm          34826.0  44204.0  
gemver            NaN  56083.0  
gesummv       21070.0  35794.0  
mvt               NaN   5005.0  
symm              NaN      NaN  
syr2k         16273.0  50119.0  
syrk          13308.0  36197.0  
three_mm      23121.0  59475.0  
trmm          35494.0  19090.0  
two_mm        47079.0  47349.0


DSPs


opt_strategy opt_full                     opt_nbt                   opt_none  \
k_size             2      4      8     16      2     4      8    16       2    
kernel_name                                                                    
atax              8.0   24.0   60.0  60.0     NaN   NaN    NaN  NaN      2.0   
bicg             16.0   26.0   40.0  54.0     NaN   NaN    NaN  NaN      2.0   
doitgen          24.0   48.0  138.0   NaN     NaN   NaN    NaN  NaN      2.0   
gemm             16.0   72.0  152.0   NaN     NaN   NaN    NaN  NaN      8.0   
gemver           16.0   64.0  240.0   NaN     NaN   NaN    NaN  NaN      4.0   
gesummv          16.0   24.0   32.0  40.0     NaN   NaN    NaN  NaN      2.0   
mvt              16.0   26.0   40.0  54.0     NaN   NaN    NaN  NaN      2.0   
symm              NaN    NaN    NaN   NaN    10.0  20.0  136.0  NaN      4.0   
syr2k            22.0   18.0   26.0  32.0     NaN   NaN    NaN  NaN      6.0   
syrk             14.0   10.0   14.0  14.0     NaN   NaN    NaN  NaN      6.0   
three_mm         32.0  128.0    NaN   NaN     NaN   NaN    NaN  NaN      2.0   
trmm              NaN    NaN    NaN   NaN     8.0   8.0    2.0  2.0      6.0   
two_mm           16.0  128.0  704.0   NaN     NaN   NaN    NaN  NaN      8.0   

opt_strategy                scalehls                       
k_size         4    8    16       2      4      8      16  
kernel_name                                                
atax          2.0  2.0  2.0      NaN    NaN    NaN   40.0  
bicg          2.0  2.0  2.0      5.0   10.0   15.0   30.0  
doitgen       2.0  2.0  2.0     40.0  215.0  245.0  245.0  
gemm          8.0  2.0  2.0     32.0  208.0  184.0  249.0  
gemver        4.0  4.0  4.0      NaN    NaN    NaN  215.0  
gesummv       2.0  2.0  4.0     28.0   96.0  118.0  153.0  
mvt           2.0  2.0  2.0      NaN    NaN    NaN   20.0  
symm          4.0  4.0  4.0      NaN    NaN    NaN    NaN  
syr2k         4.0  4.0  4.0      8.0   22.0   78.0  193.0  
syrk          2.0  2.0  2.0     11.0   25.0   65.0  134.0  
three_mm      2.0  2.0  2.0      NaN    NaN   50.0  257.0  
trmm          8.0  2.0  2.0     21.0  124.0  204.0   88.0  
two_mm        8.0  2.0  2.0      NaN    NaN  230.0  184.0


Power


opt_strategy opt_full                      opt_nbt                       \
k_size             2      4      8      16      2      4      8      16   
kernel_name                                                               
atax            0.349  0.443  0.707  1.288     NaN    NaN    NaN    NaN   
bicg            0.371  0.392  0.695  1.250     NaN    NaN    NaN    NaN   
doitgen         0.404  0.620  1.815    NaN     NaN    NaN    NaN    NaN   
gemm            0.377  0.468  1.818    NaN     NaN    NaN    NaN    NaN   
gemver          0.412  0.660  1.313    NaN     NaN    NaN    NaN    NaN   
gesummv         0.376  0.418  0.667  1.116     NaN    NaN    NaN    NaN   
mvt             0.372  0.427  0.690  1.255     NaN    NaN    NaN    NaN   
symm              NaN    NaN    NaN    NaN   0.360  0.395  1.634    NaN   
syr2k           0.390  0.361  0.394  0.792     NaN    NaN    NaN    NaN   
syrk            0.366  0.346  0.368  0.523     NaN    NaN    NaN    NaN   
three_mm        0.433  1.139    NaN    NaN     NaN    NaN    NaN    NaN   
trmm              NaN    NaN    NaN    NaN   0.345  0.367  0.336  0.336   
two_mm          0.376  0.900  4.116    NaN     NaN    NaN    NaN    NaN   

opt_strategy opt_none                      scalehls              
k_size             2      4      8      16       2   4   8   16  
kernel_name                                                      
atax            0.347  0.351  0.360  0.384      NaN NaN NaN NaN  
bicg            0.348  0.352  0.366  0.338      NaN NaN NaN NaN  
doitgen         0.363  0.343  0.338  0.344      NaN NaN NaN NaN  
gemm            0.364  0.383  0.341  0.345      NaN NaN NaN NaN  
gemver          0.368  0.413  0.408  0.401      NaN NaN NaN NaN  
gesummv         0.352  0.353  0.365  0.334      NaN NaN NaN NaN  
mvt             0.350  0.385  0.369  0.402      NaN NaN NaN NaN  
symm            0.371  0.346  0.349  0.340      NaN NaN NaN NaN  
syr2k           0.364  0.338  0.348  0.339      NaN NaN NaN NaN  
syrk            0.354  0.336  0.340  0.341      NaN NaN NaN NaN  
three_mm        0.371  0.412  0.351  0.337      NaN NaN NaN NaN  
trmm            0.334  0.373  0.339  0.338      NaN NaN NaN NaN  
two_mm          0.375  0.407  0.344  0.340      NaN NaN NaN NaN

In [68]:
print("Cycles")

df_pivoted=df.pivot(index='kernel_name', columns=[ 'opt_strategy', 'k_size'], values='cycles').sort_index(axis='columns', level=['opt_strategy','k_size'])
df_pivoted

Cycles


opt_strategy opt_full                        opt_nbt                          \
k_size             2      4       8       16      2      4       8        16   
kernel_name                                                                    
atax             21.0   34.0    60.0   157.0     NaN    NaN     NaN      NaN   
bicg             13.0   28.0    45.0   141.0     NaN    NaN     NaN      NaN   
doitgen          15.0  166.0  1158.0     NaN     NaN    NaN     NaN      NaN   
gemm             16.0   28.0    71.0     NaN     NaN    NaN     NaN      NaN   
gemver           36.0   49.0    75.0     NaN     NaN    NaN     NaN      NaN   
gesummv          18.0   25.0    46.0   142.0     NaN    NaN     NaN      NaN   
mvt              13.0   21.0    45.0   141.0     NaN    NaN     NaN      NaN   
symm              NaN    NaN     NaN     NaN    15.0   18.0   267.0      NaN   
syr2k            19.0  270.0  1417.0  8835.0     NaN    NaN     NaN      NaN   
syrk             16.0  222.0  1211.0  7896.0     NaN    NaN     NaN      NaN   
three_mm         22.0   40.0     NaN     NaN     NaN    NaN     NaN      NaN   
trmm              NaN    NaN     NaN     NaN    12.0  288.0  2138.0  15730.0   
two_mm           25.0   43.0    98.0     NaN     NaN    NaN     NaN      NaN   

opt_strategy opt_none                            scalehls                       
k_size             2       4        8         16       2     4      8       16  
kernel_name                                                                     
atax             76.0   299.0   1171.0    4643.0      NaN   NaN    NaN  1701.0  
bicg             73.0   294.0   1162.0    4626.0     26.0  43.0   78.0   151.0  
doitgen         165.0  2486.0  38986.0  344338.0     19.0  33.0   96.0    96.0  
gemm            103.0   794.0   6538.0   42514.0     23.0  33.0   64.0   146.0  
gemver          154.0   606.0   2377.0    9620.0      NaN   NaN    NaN   242.0  
gesummv          92.0   326.0   1226.0    5026.0     28.0  38.0   65.0   111.0  
mvt              74.0   290.0   1155.0    4611.0      NaN   NaN    NaN   112.0  
symm            229.0  1593.0      NaN   90133.0      NaN   NaN    NaN     NaN  
syr2k            99.0   706.0   4834.0   35650.0     35.0  47.0   81.0   381.0  
syrk             79.0   546.0   3682.0   22594.0     28.0  39.0   74.0   340.0  
three_mm        220.0  1743.0  14042.0  111410.0      NaN   NaN  188.0   827.0  
trmm             26.0   301.0   2138.0   15730.0     40.0  84.0  182.0  1199.0  
two_mm          176.0  1375.0  11218.0   87842.0      NaN   NaN  124.0   788.0

In [69]:
top='opt_none'
bot='opt_full'
print("Speedup - {}/{}".format(top,bot))
tmp = (df_pivoted[top]/df_pivoted[bot]).filter([2,4,8,16])
display(tmp)
print("Mean of all values is: {:.2f}".format(tmp.stack().mean()))

Speedup - opt_none/opt_full


k_size,2,4,8,16
kernel_name,,,,
atax,3.619048,8.794118,19.516667,29.573248
bicg,5.615385,10.500000,25.822222,32.808511
doitgen,11.000000,14.975904,33.666667,NaN
gemm,6.437500,28.357143,92.084507,NaN
gemver,4.277778,12.367347,31.693333,NaN
gesummv,5.111111,13.040000,26.652174,35.394366
mvt,5.692308,13.809524,25.666667,32.702128
symm,NaN,NaN,NaN,NaN
syr2k,5.210526,2.614815,3.411433,4.035088


Mean of all values is: 20.13


In [70]:
complete=df_pivoted['opt_full'].fillna(df_pivoted['opt_nbt'])
top='opt_none'
bot='opt_all'
print("Speedup - {}/{}".format(top,bot))
tmp = (df_pivoted[top]/complete).filter([2,4,8,16])
display(tmp)
print("Mean of all values is: {:.2f}".format(tmp.stack().mean()))


# tmp.to_csv('../results/sodaopt_speedup.csv')

Speedup - opt_none/opt_all


k_size,2,4,8,16
kernel_name,,,,
atax,3.619048,8.794118,19.516667,29.573248
bicg,5.615385,10.500000,25.822222,32.808511
doitgen,11.000000,14.975904,33.666667,NaN
gemm,6.437500,28.357143,92.084507,NaN
gemver,4.277778,12.367347,31.693333,NaN
gesummv,5.111111,13.040000,26.652174,35.394366
mvt,5.692308,13.809524,25.666667,32.702128
symm,15.266667,88.500000,NaN,NaN
syr2k,5.210526,2.614815,3.411433,4.035088


Mean of all values is: 19.86


In [71]:
top='opt_none'
bot='opt_nbt'
print("Speedup - {}/{}".format(top,bot))
tmp = (df_pivoted[top]/df_pivoted[bot]).filter([2,4,8,16])
display(tmp)
print("Mean of all values is: {:.2f}".format(tmp.stack().mean()))

Speedup - opt_none/opt_nbt


k_size,2,4,8,16
kernel_name,,,,
atax,NaN,NaN,NaN,NaN
bicg,NaN,NaN,NaN,NaN
doitgen,NaN,NaN,NaN,NaN
gemm,NaN,NaN,NaN,NaN
gemver,NaN,NaN,NaN,NaN
gesummv,NaN,NaN,NaN,NaN
mvt,NaN,NaN,NaN,NaN
symm,15.266667,88.500000,NaN,NaN
syr2k,NaN,NaN,NaN,NaN


Mean of all values is: 18.16


In [72]:
top='scalehls'
bot='opt_full'
print("Speedup - {}/{}".format(top,bot))
tmp = (df_pivoted[top]/df_pivoted[bot]).filter([2,4,8,16])
display(tmp)
print("Mean of all values is: {:.2f}".format(tmp.stack().mean()))

Speedup - scalehls/opt_full


k_size,2,4,8,16
kernel_name,,,,
atax,NaN,NaN,NaN,10.834395
bicg,2.000000,1.535714,1.733333,1.070922
doitgen,1.266667,0.198795,0.082902,NaN
gemm,1.437500,1.178571,0.901408,NaN
gemver,NaN,NaN,NaN,NaN
gesummv,1.555556,1.520000,1.413043,0.781690
mvt,NaN,NaN,NaN,0.794326
symm,NaN,NaN,NaN,NaN
syr2k,1.842105,0.174074,0.057163,0.043124


Mean of all values is: 1.35


In [73]:
# df_pivoted.plot(kind='bar')

In [74]:
# df.pivot(index=['kernel_name'],columns=['opt_strategy'],values=['cycles'])
# df.columns

# df.plot(x=['kernel_name','k_size','opt_strategy'], y='cycles', kind='bar')
# df.plot(x=['kernel_name','k_size','opt_strategy'], y='cycles', kind='bar')
# df.plot(x='kernel_name', y='cycles', kind='bar')
